<a href="https://colab.research.google.com/github/damianmtz98/PyCdeD/blob/main/T8_ReconocimientoComandodeVoz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pywt
from sklearn.model_selection import train_test_split
import librosa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Dropout
import matplotlib.pyplot as plt


## Carga de los datos de audio


In [ ]:
# descargamos el conjunto de datos mini_speech_commands
ds, info = tfds.load('speech_commands', with_info=True, as_supervised=True, split=['train[:10%]', 'test[:10%]'])


In [ ]:
# lista de los comandos
commands = info.features['label'].names
print(f"Comandos disponibles: {commands}")


In [ ]:
# función para extraer características de wavelet de una señal de audio
def extract_wavelet_features(signal, wavelet='db1', level=6):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    features = np.concatenate(coeffs)
    return features

In [ ]:
# función para extraer una muestra de cada comando del dataset y visualizar
def extract_and_plot_all_samples(dataset, commands, target_length=16000):
    # convertirmos el dataset a formato numpy
    dataset = tfds.as_numpy(dataset)
    command_samples = {command: None for command in commands}

    # extraemos una muestra de cada comando
    for audio, label in dataset:
        command = commands[label]
        if command_samples[command] is None:
            signal = audio
            if len(signal) < target_length:
                signal = np.pad(signal, (0, target_length - len(signal)), 'constant')
            else:
                signal = signal[:target_length]
            command_samples[command] = signal

    # visualizamos las muestras
    for command, signal in command_samples.items():
        plot_waveform(signal, f'Forma de onda del comando: {command}')

In [ ]:

def plot_waveform(signal, title, sample_rate=16000):
    plt.figure(figsize=(12, 4))
    plt.title(title)
    plt.xlabel('Tiempo')
    plt.ylabel('Amplitud')
    plt.plot(signal)
    plt.show()

In [ ]:
# cargamos el conjunto de datos de los comandos de voz
ds, info = tfds.load('speech_commands', with_info=True, as_supervised=True, split='train[:10%]')
commands = info.features['label'].names
print(f"Comandos disponibles: {commands}")

In [ ]:
# extraemos y visualizamos una muestra de cada comando de voz
extract_and_plot_all_samples(ds, commands)

In [ ]:
# función para preprocesar los datos de audio
def preprocess_dataset(dataset, target_length=16000):
    data = []
    labels = []
    for audio, label in tfds.as_numpy(dataset):
        # ajustamos la longitud de la señal de audio
        if len(audio) < target_length:
            audio = np.pad(audio, (0, target_length - len(audio)), 'constant')
        else:
            audio = audio[:target_length]

        wavelet_features = extract_wavelet_features(audio)
        data.append(wavelet_features)
        labels.append(label)

    return np.array(data), np.array(labels)

## Preprocesamiento

In [ ]:
# preprocesamos los datos de entrenamiento y prueba
X_train, y_train = preprocess_dataset(ds[0])
X_test, y_test = preprocess_dataset(ds[1])


In [ ]:
# verificamos las formas de los datos
print(f"conjunto de datos entrenamiento: {X_train.shape}")
print(f"conjunto de datos prueba: {X_test.shape}")

In [ ]:
# redefinimos la forma de los datos de comando para el modelo
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]

## Modelo

In [ ]:
# definimos el modelo
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(commands), activation='softmax')
])

In [ ]:
# definimos el modelo
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(commands), activation='softmax')
])

In [ ]:
# compilamos el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# entrenamos el modelo
history = model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
# evaluamos el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

In [ ]:
# graficamos la pérdida y la precision durante el entrenamiento y la validación
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Pérdida durante el entrenamiento y la validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Precisión de entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de validación')
plt.title('Precisión durante el entrenamiento y la validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()
